## Enrichment

### Introduction

We call enrichment to the process of augmenting your data with new variables by means of a spatial join between your data and a `Dataset` aggregated at a given spatial resolution in the CARTO Data Observatory or in other words:

"*Enrichment is the process of adding variables to a geometry, which we call the target, (point, line, polygon…) from a spatial (polygon) dataset, which we call the source*"

We recommend you to check the [CARTOframes quickstart](https://carto.com/developers/cartoframes/guides/Quickstart/) since we'll use some of the DataFrames generated in the exaple and the [Discovery guide](https://carto.com/developers/cartoframes/guides/Data-discovery) to learn how to explore the Data Observatory catalog to find out variables of interest for your analyses.

### Choose variables to enrich from the Data Observatory catalog

Let's follow up the [Discovery guide](https://carto.com/developers/cartoframes/guides/Data-discovery) where we subscribed to the AGS demographics dataset and list the variables we have available to enrich our own data.

In [1]:
from cartoframes.auth import set_default_credentials
set_default_credentials(username= 'carto-do-1', api_key='3f1605d074dd2b52cebb2e3bbcfe4a0565ff1660')

In [2]:
from cartoframes.data.observatory import Catalog, Dataset, Variable, Geography
Catalog().subscriptions().datasets

[<Dataset.get('ags_sociodemogr_e92b1637')>]

In [3]:
dataset = Dataset.get('ags_sociodemogr_e92b1637')
variables = dataset.variables
variables

[<Variable.get('HINCYMED65_310bc888')> #'Median Household Income: Age 65-74 (2019A)',
 <Variable.get('HINCYMED55_1a269b4b')> #'Median Household Income: Age 55-64 (2019A)',
 <Variable.get('HINCYMED45_33daa0a')> #'Median Household Income: Age 45-54 (2019A)',
 <Variable.get('HINCYMED35_4c7c3ccd')> #'Median Household Income: Age 35-44 (2019A)',
 <Variable.get('HINCYMED25_55670d8c')> #'Median Household Income: Age 25-34 (2019A)',
 <Variable.get('HINCYMED24_22603d1a')> #'Median Household Income: Age < 25 (2019A)',
 <Variable.get('HINCYGT200_e552a738')> #'Household Income > $200000 (2019A)',
 <Variable.get('HINCY6075_1933e114')> #'Household Income $60000-$74999 (2019A)',
 <Variable.get('HINCY4550_f7ad7d79')> #'Household Income $45000-$49999 (2019A)',
 <Variable.get('HINCY4045_98177a5c')> #'Household Income $40000-$44999 (2019A)',
 <Variable.get('HINCY3540_73617481')> #'Household Income $35000-$39999 (2019A)',
 <Variable.get('HINCY2530_849c8523')> #'Household Income $25000-$29999 (2019A)',
 <V

As we saw in the discovery guide the `ags_sociodemogr_e92b1637` dataset contains socio-demographic variables aggregated at the Census blockgroup level. Let's try to find out a variable indicating the total population.

In [4]:
vdf = variables.to_dataframe()
vdf[vdf['name'].str.contains('pop', case=False, na=False)]

,agg_method,column_name,dataset_id,db_type,description,id,name,slug,starred,summary_json,variable_group_id
55,SUM,POPPY,carto-do.ags.demographics_sociodemographic_usa...,FLOAT,Population (2024A),carto-do.ags.demographics_sociodemographic_usa...,POPPY,POPPY_946f4ed6,False,"{'head': [0, 0, 8, 0, 0, 0, 4, 0, 2, 59], 'tai...",None
75,SUM,POPCYGRPI,carto-do.ags.demographics_sociodemographic_usa...,INTEGER,Institutional Group Quarters Population (2019A),carto-do.ags.demographics_sociodemographic_usa...,POPCYGRPI,POPCYGRPI_147af7a9,False,"{'head': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'tail...",None
76,SUM,POPCYGRP,carto-do.ags.demographics_sociodemographic_usa...,INTEGER,Population in Group Quarters (2019A),carto-do.ags.demographics_sociodemographic_usa...,POPCYGRP,POPCYGRP_74c19673,False,"{'head': [0, 0, 0, 0, 959, 0, 0, 0, 0, 0], 'ta...",None
77,SUM,POPCY,carto-do.ags.demographics_sociodemographic_usa...,INTEGER,Population (2019A),carto-do.ags.demographics_sociodemographic_usa...,POPCY,POPCY_f5800f44,False,"{'head': [6, 5, 0, 22, 959, 0, 0, 0, 0, 0], 't...",None
99,SUM,LBFCYPOP16,carto-do.ags.demographics_sociodemographic_usa...,INTEGER,Population Age 16+ (2019A),carto-do.ags.demographics_sociodemographic_usa...,LBFCYPOP16,LBFCYPOP16_53fa921c,False,"{'head': [6, 3, 0, 20, 959, 0, 0, 0, 0, 0], 't...",None


We can store the variable instance we need by searching the Catalog by its `slug`, in this case `POPCY_f5800f44`:

In [5]:
variable = Variable.get('POPCY_f5800f44')
variable.to_dict()

{'id': 'carto-do.ags.demographics_sociodemographic_usa_blockgroup_2015_yearly_2019.POPCY',
 'slug': 'POPCY_f5800f44',
 'name': 'POPCY',
 'description': 'Population (2019A)',
 'column_name': 'POPCY',
 'db_type': 'INTEGER',
 'dataset_id': 'carto-do.ags.demographics_sociodemographic_usa_blockgroup_2015_yearly_2019',
 'agg_method': 'SUM',
 'variable_group_id': None,
 'starred': False}

The `POPCY` variable contains the `SUM` of the population for blockgroup for the year 2019. Let's enrich our stores DataFrame with that variable.

### Enrich a points DataFrame

We learned in the [CARTOframes quickstart](https://carto.com/developers/cartoframes/guides/Quickstart/) how to load our own data (in this case Starbucks stores) and geocode their addresses to coordinates for further analysis, so we start by loading our geocoded Starbucks stores:

In [6]:
import geopandas as gpd
stores_df = gpd.read_file('../files/starbucks_brooklyn_geocoded.geojson')
stores_df.head(5)

,cartodb_id,field_1,name,address,revenue,geometry
0,1,0,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772,POINT (-73.95901 40.67109)
1,2,1,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418,POINT (-73.96122 40.57796)
2,3,2,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699,POINT (-73.98976 40.61912)
3,4,3,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676,POINT (-74.02744 40.63152)
4,5,4,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411,POINT (-74.00098 40.59321)


**Note: We could alternatively load any geospatial format supported by GeoPandas or CARTO. See the Data Management guide for more information about this**

As we can see for each store we have its name, address, the total revenue by year and a `geometry` column indicating the location of the store. This is important because for the enrichment service to work we need a DataFrame with a geometry column encoded as a [shapely](https://pypi.org/project/Shapely/) object.

We can now create a new `Enrichment` instance, and since the `stores_df` dataset represents stores locations (points), we can use the `enrich_points` function passing as arguments, the stores DataFrame and a list of `Variables` we have a valid subscription from the Data Observatory catalog.

In this case we are just enriching one variable (the total population), but we could pass a list of them.

In [7]:
from cartoframes.data.observatory import Enrichment
enriched_stores_df = Enrichment().enrich_points(stores_df, [variable])
enriched_stores_df.head(5)

,cartodb_id,field_1,name,address,revenue,geometry,POPCY,do_geom_area
0,1,0,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772,POINT (-73.95901 40.67109),2215,59840.196748
1,2,1,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418,POINT (-73.96122 40.57796),1831,60150.636995
2,3,2,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699,POINT (-73.98976 40.61912),745,38950.618837
3,4,3,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676,POINT (-74.02744 40.63152),1174,57353.293114
4,5,4,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411,POINT (-74.00098 40.59321),2289,188379.242640


Once the enrichment finishes, we've obtained a new column in our DataFrame called `POPCY` with the `SUM` of population projected for the year 2019, in the US Census blockgroup which contains each one of our Starbucks stores.

The reason why we are obtaining the `SUM` is because we are using the `ags_sociodemogr_e92b1637` which data is aggregated at the Census blockgroup level and more concretely the `POPCY` variable is aggregated by `SUM` as we can see in the Catalog `Variable` metadata:

In [10]:
variable.agg_method

'SUM'

All this information, is available in the `ags_sociodemogr_e92b1637` metadata. Let's take a look:

In [11]:
dataset.to_dict()

{'id': 'carto-do.ags.demographics_sociodemographic_usa_blockgroup_2015_yearly_2019',
 'slug': 'ags_sociodemogr_e92b1637',
 'name': 'Sociodemographic',
 'description': 'Census and ACS sociodemographic data estimated for the current year and data projected to five years. Projected fields are general aggregates (total population, total households, median age, avg income etc.)',
 'country_id': 'usa',
 'geography_id': 'carto-do.ags.geography_usa_blockgroup_2015',
 'geography_name': 'USA Census Block Group',
 'geography_description': None,
 'category_id': 'demographics',
 'category_name': 'Demographics',
 'provider_id': 'ags',
 'provider_name': 'Applied Geographic Solutions',
 'data_source_id': 'sociodemographic',
 'lang': 'eng',
 'temporal_aggregation': 'yearly',
 'time_coverage': '[2019-01-01,2020-01-01)',
 'update_frequency': None,
 'version': '2019',
 'is_public_data': False}

### Enrich a polygons DataFrame

Let's do a second enrichment but in this case let's use the DataFrame with the areas of influence calculated in the [Quickstart guide](https://carto.com/developers/cartoframes/guides/Quickstart-Part-1/). There, we used the [CARTOframes isochrones](https://carto.com/developers/cartoframes/reference/#heading-Isolines) service to obtain the polygon around each store that cover the area within 8, 17 and 25 minutes walk.

In [12]:
aoi_df = gpd.read_file('../files/starbucks_brooklyn_isolines.geojson')
aoi_df.head(5)

,data_range,lower_data_range,range_label,geometry
0,500,0,8 min.,"MULTIPOLYGON (((-73.95959 40.67571, -73.95971 ..."
1,1000,500,17 min.,"POLYGON ((-73.95988 40.68110, -73.95863 40.681..."
2,1500,1000,25 min.,"POLYGON ((-73.95986 40.68815, -73.95711 40.688..."
3,500,0,8 min.,"MULTIPOLYGON (((-73.96185 40.58321, -73.96231 ..."
4,1000,500,17 min.,"MULTIPOLYGON (((-73.96684 40.57483, -73.96830 ..."


In this case we have a DataFrame which, for each index in the `stores_df` contains a polygon of the areas of influence around each store at 8, 17 and 25 minutes walking. Again the `geometry` is encoded as a `shapely` object.

In this case, the `Enrichment` service provides an `enrich_polygons` function, which in his basic version, works in the same way as the `enrich_points` function. It just needs a DataFrame with a polygon geometry and a list of variables to enrich:

In [13]:
from cartoframes.data.observatory import Enrichment
enriched_aoi_df = Enrichment().enrich_polygons(aoi_df, [variable])
enriched_aoi_df.head(5)

,data_range,lower_data_range,range_label,geometry,sum_POPCY
0,500,0,8 min.,"MULTIPOLYGON (((-73.95959 40.67571, -73.95971 ...",1278.961116
1,1000,500,17 min.,"POLYGON ((-73.95988 40.68110, -73.95863 40.681...",1159.707618
2,1500,1000,25 min.,"POLYGON ((-73.95986 40.68815, -73.95711 40.688...",997.870053
3,500,0,8 min.,"MULTIPOLYGON (((-73.96185 40.58321, -73.96231 ...",1628.693179
4,1000,500,17 min.,"MULTIPOLYGON (((-73.96684 40.57483, -73.96830 ...",1439.783746


We have obtained a new column in our areas of influence DataFrame, `SUM_POPCY` which represents the `SUM` of total population in the Census blockgroups that instersect with each polygon in our DataFrame.

### How enrichment works

Let's try to explain what happens under the hood when you do a polygons enrichment.

Imagine we have polygons representing municipalities, in blue, each of which have a population attribute, and we want to find out the population inside the green circle. 

<img src="../files/enrichment_01.png" width="400"/>

We don’t know how the population is distributed inside these municipalities. They are probably concentrated in cities somewhere but, since we don’t know where those are, our best guess is to assume that the population is evenly distributed in the municipality (every point inside the municipality has the same population density).

Population is an extensive property (it grows with area), so we can subset it (a region inside the municipality will always have a smaller population than the whole municipality), and also aggregate it by summing.

In this case, we’d calculate the population inside each part of the circle that intersects with a municipality.

**Default aggregation methods**

In the Data Observatory, we suggest a default aggregation method for certain fields (always weighted by intersected area). However, some fields don’t have a clear best method, and some just can’t be aggregated. In these cases, we leave the agg_method field blank and let the users choose the method that fits best for their needs.

### Conclusion

In this guide we've learned how to use CARTOframes together with the Data Observatory to enrich our Starbucks dataset with a new population variable for our use case of revenue prediction analysis. For that purpose we've:

- Choose the total population variable from the Data Observatory catalog
- Calculated the sum of total population for each store
- Calculated the sum of total population around the walking areas of influence around each store

Finally we've introduced some other advanced concepts and further explanation on how the enrichment works.